In [1]:
import pandas as pd
import numpy as np

# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return. 

# Read in the airports data.
airports = pd.read_csv("data/airports.dat", header=None, na_values='\\N')
airports.columns = ["id", "name", "city", "country", "iata", "icao", "latitude", "longitude", "altitude","timezone", "dst", "tz", "type", "source"]

# Read in the airlines data.
airlines = pd.read_csv("data/airlines.dat", header=None, na_values='\\N')
airlines.columns = ["id", "name", "alias", "iata", "icao", "callsign", "country", "active"]

# Read in the routes data.
routes = pd.read_csv("data/routes.dat", header=None, na_values='\\N')
routes.columns = ["airline", "airline_id", "source", "source_id", "dest", "dest_id", "codeshare", "stops", "equipment"]

In [2]:
# Select the LEX routes, then join the source airports
lex_routes = routes[(routes['source']=="LEX")]
lex_routes = pd.merge(lex_routes, airports, left_on='source_id', right_on='id', how='left')
lex_routes
lex_routes.to_excel("lex_step1.xlsx")

In [3]:
# join the destination airports.  Here we need to use the suffixes option, because 
# the column names overlap, and we want to distinguish between source and dest
lex_routes = pd.merge(lex_routes, airports, 
                      left_on='dest_id', 
                      right_on='id', 
                      how='left', 
                      suffixes=['_source','_dest'])
lex_routes.to_excel("lex_step2.xlsx")

In [4]:
# It looks like source has some duplicate names.  Drop the values from the airports
# file and keep the one from the routes file
lex_routes = lex_routes.drop(['source_y','source'], axis=1)
lex_routes = lex_routes.rename(columns={'source_x': 'source'})
lex_routes.to_excel("lex_step3.xlsx")

In [5]:
# Let's keep only one route between each airport pair
# so we don't have a bunch of lines on top of each other
# The subset option tells it to consider just those columns when determining
# what is a duplicate. 
lex_routes = lex_routes.drop_duplicates(subset=['source', 'dest'])
lex_routes.to_excel("lex_step4.xlsx")
#lex_routes

In [6]:
lex_routes2 = pd.merge(lex_routes, routes, 
                      left_on='dest_id', 
                      right_on='source_id', 
                      how='left', 
                      suffixes=['_source2','_dest2'])
lex_routes2

,airline_source2,airline_id_source2,source_source2,source_id_source2,dest_source2,dest_id_source2,codeshare_source2,stops_source2,equipment_source2,id_source,...,type_dest,airline_dest2,airline_id_dest2,source_dest2,source_id_dest2,dest_dest2,dest_id_dest2,codeshare_dest2,stops_dest2,equipment_dest2
0,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,airport,3M,20710.0,ATL,3682.0,LWB,6958.0,NaN,0,SF3
1,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,airport,3M,20710.0,ATL,3682.0,MCN,3754.0,NaN,0,SF3
2,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,airport,3M,20710.0,ATL,3682.0,MEI,4335.0,NaN,0,SF3
3,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,airport,3M,20710.0,ATL,3682.0,MSL,5756.0,NaN,0,SF3
4,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,airport,3M,20710.0,ATL,3682.0,PIB,5759.0,NaN,0,SF3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3553,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,4017,...,airport,US,5265.0,IAH,3550.0,PHX,3462.0,NaN,0,319 320
3554,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,4017,...,airport,US,5265.0,IAH,3550.0,SAL,1892.0,NaN,0,E90
3555,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,4017,...,airport,VA,5360.0,IAH,3550.0,ATL,3682.0,Y,0,M88 717 319
3556,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,4017,...,airport,VB,10646.0,IAH,3550.0,MTY,1825.0,NaN,0,733


In [7]:
lex_routes2 = pd.merge(lex_routes2, airports, 
                      left_on='dest_id_dest2', 
                      right_on='id', 
                      how='left')
lex_routes2

,airline_source2,airline_id_source2,source_source2,source_id_source2,dest_source2,dest_id_source2,codeshare_source2,stops_source2,equipment_source2,id_source,...,iata,icao,latitude,longitude,altitude,timezone,dst,tz,type,source
0,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,LWB,KLWB,37.858299,-80.399498,2302.0,-5.0,U,America/New_York,airport,OurAirports
1,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,MCN,KMCN,32.692799,-83.649200,354.0,-5.0,A,America/New_York,airport,OurAirports
2,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,MEI,KMEI,32.332600,-88.751900,297.0,-6.0,A,America/Chicago,airport,OurAirports
3,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,MSL,KMSL,34.745300,-87.610199,551.0,-6.0,A,America/Chicago,airport,OurAirports
4,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,PIB,KPIB,31.467100,-89.337097,298.0,-6.0,A,America/Chicago,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3553,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,4017,...,PHX,KPHX,33.434299,-112.012001,1135.0,-7.0,N,America/Phoenix,airport,OurAirports
3554,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,4017,...,SAL,MSLP,13.440900,-89.055702,101.0,-6.0,U,America/El_Salvador,airport,OurAirports
3555,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,4017,...,ATL,KATL,33.636700,-84.428101,1026.0,-5.0,A,America/New_York,airport,OurAirports
3556,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,4017,...,MTY,MMMY,25.778500,-100.107002,1278.0,-6.0,S,America/Mexico_City,airport,OurAirports


In [8]:
lex_routes2 = lex_routes2.drop_duplicates(subset=['source_dest2', 'dest_dest2'])
lex_routes2

,airline_source2,airline_id_source2,source_source2,source_id_source2,dest_source2,dest_id_source2,codeshare_source2,stops_source2,equipment_source2,id_source,...,iata,icao,latitude,longitude,altitude,timezone,dst,tz,type,source
0,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,LWB,KLWB,37.858299,-80.399498,2302.0,-5.0,U,America/New_York,airport,OurAirports
1,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,MCN,KMCN,32.692799,-83.649200,354.0,-5.0,A,America/New_York,airport,OurAirports
2,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,MEI,KMEI,32.332600,-88.751900,297.0,-6.0,A,America/Chicago,airport,OurAirports
3,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,MSL,KMSL,34.745300,-87.610199,551.0,-6.0,A,America/Chicago,airport,OurAirports
4,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,PIB,KPIB,31.467100,-89.337097,298.0,-6.0,A,America/Chicago,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,4017,...,VSA,MMVA,17.997000,-92.817398,46.0,-6.0,S,America/Mexico_City,airport,OurAirports
3536,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,4017,...,XNA,KXNA,36.281898,-94.306801,1287.0,-6.0,A,America/Chicago,airport,OurAirports
3539,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,4017,...,YVR,CYVR,49.193901,-123.183998,14.0,-8.0,A,America/Vancouver,airport,OurAirports
3542,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,4017,...,ZIH,MMZH,17.601601,-101.460999,26.0,-6.0,S,America/Mexico_City,airport,OurAirports


In [9]:
# create a basic map, centered on Lexington
import folium
lex_air = folium.Map(
    location=[38.034,-84.500],
    tiles='Stamen Toner',
    zoom_start=4
)

In [10]:
#Define some empty sets
airport_set = set()
route_set = set()

# Make sure we don't add duplicates, especially for the origins
for name, row in lex_routes.iterrows():
    
    if row['source'] not in airport_set: #create circle marker to the airport
        popup_string = row['city_source'] + ' (' + row['source'] + ')'
        marker = folium.CircleMarker([row["latitude_source"], row["longitude_source"]], 
                                     color='DarkCyan',
                                     fill_color='DarkCyan', 
                                     radius=5, popup=popup_string)
        marker.add_to(lex_air)
        airport_set.add(row['source'])
        
    if row['dest'] not in airport_set: 
        popup_string = row['city_dest'] + '(' + row['dest'] + ')'
        marker = folium.CircleMarker([row["latitude_dest"], row["longitude_dest"]], 
                                     color='MidnightBlue',
                                     fill_color='MidnightBlue', 
                                     radius=5, popup=popup_string)
        marker.add_to(lex_air) #add it to map
        airport_set.add(row['dest'])
    
    # the parentheses in the indicate that we are adding a tuple to the route_set
    if (row['source'],row['dest']) not in route_set:            
        popup_string = row['source'] + '-' + row['dest']        
        line = folium.PolyLine([(row["latitude_source"], row["longitude_source"]), 
                                (row["latitude_dest"], row["longitude_dest"])], 
                                weight=2, 
                                popup=popup_string)
        line.add_to(lex_air)
        route_set.add((row['source'],row['dest']))
        
lex_air

In [11]:
lex_routes2=lex_routes2.dropna()

In [12]:
#Define some empty sets
airport_set = set()
route_set = set()

# Make sure we don't add duplicates, especially for the origins
for name, row in lex_routes2.iterrows():
    
    if row['source_dest2'] not in airport_set: #create circle marker to the airport
        popup_string = row['city_dest'] + ' (' + row['source_dest2'] + ')'
        marker = folium.CircleMarker([row["latitude_dest"], row["longitude_dest"]], 
                                     color='DarkCyan',
                                     fill_color='DarkCyan', 
                                     radius=5, popup=popup_string)
        marker.add_to(lex_air)
        airport_set.add(row['source_dest2'])
        
    if row['dest_dest2'] not in airport_set: 
        popup_string = row['city'] + '(' + row['dest_dest2'] + ')'
        marker = folium.CircleMarker([row["latitude"], row["longitude"]], 
                                     color='MidnightBlue',
                                     fill_color='MidnightBlue', 
                                     radius=5, popup=popup_string)
        marker.add_to(lex_air) #add it to map
        airport_set.add(row['dest_dest2'])
    
    # the parentheses in the indicate that we are adding a tuple to the route_set
    if (row['source_dest2'],row['dest_dest2']) not in route_set:            
        popup_string = row['source_dest2'] + '-' + row['dest_dest2']        
        line = folium.PolyLine([(row["latitude_dest"], row["longitude_dest"]), 
                                (row["latitude"], row["longitude"])], 
                                weight=2, 
                                popup=popup_string)
        line.add_to(lex_air)
        route_set.add((row['source_dest2'],row['dest_dest2']))
        
lex_air